<a href="https://colab.research.google.com/github/akhileshchoudary/bone-fracture-classification/blob/main/Bone_Break_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
# I mount my Google Drive to access the dataset stored in the cloud.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
# I define the path to the dataset directory for easy reference throughout the code.
!ls "/content/drive/My Drive/Bone Break Classification"

'Avulsion fracture'	'Hairline Fracture'	 'Pathological fracture'
'Comminuted fracture'	'Impacted fracture'	 'Spiral Fracture'
'Fracture Dislocation'	'Longitudinal fracture'
'Greenstick fracture'	'Oblique fracture'


In [10]:
data_dir = "/content/drive/My Drive/Bone Break Classification"

#### Importing Libraries

- I import the necessary libraries for data processing, visualization, and building the machine learning model.
- 'os' and 'numpy' are for file handling and numerical operations.
- 'matplotlib.pyplot' is for plotting images.
- 'tensorflow' and 'keras' provide tools for building and training the neural network.
- 'pathlib' is used for handling file paths.

In [50]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras
from keras import (
    backend as K,
    losses,
    optimizers,
    layers,
    models,
    utils,
    callbacks
)

from pathlib import Path